In [1]:
print("ok")

ok


## Download Youtube Video

In [2]:
import os
import subprocess
from pytube import YouTube
import shutil
import sys
from cine_notes.logger import logging
from cine_notes.exception import CineNotesException

class YouTubeVideoProcessor:
    """
    A class to process YouTube videos by downloading and extracting audio/frames.

    This class provides functionality to:
    - Download YouTube videos at highest available quality
    - Extract audio in WAV format
    - Extract video frames as images
    - Organize outputs in a structured directory format

    Attributes:
        youtube_url (str): URL of the YouTube video to process
        output_base_dir (str): Base directory for all outputs
        video_filename (str): Path to downloaded video file
        video_dir (str): Directory containing the video
        audio_dir (str): Directory for extracted audio
        frames_dir (str): Directory for extracted frames
    """

    def __init__(self, youtube_url: str, output_base_dir: str = "./output_videos"):
        """
        Initialize the YouTubeVideoProcessor.

        Args:
            youtube_url (str): URL of the YouTube video to process
            output_base_dir (str): Base directory for outputs (default: "./output_videos")
        """
        try:
            self.youtube_url = youtube_url
            self.output_base_dir = os.path.abspath(output_base_dir)
            os.makedirs(self.output_base_dir, exist_ok=True)
            logging.info(f"Initialized YouTubeVideoProcessor with output directory: {self.output_base_dir}")
            
            # Initialize placeholder paths
            self.video_filename = None
            self.video_dir = None
            self.audio_dir = None
            self.frames_dir = None
        except Exception as e:
            raise CineNotesException("Failed to initialize YouTubeVideoProcessor", sys)

    def download_video(self) -> str:
        """
        Download the YouTube video at highest available quality.

        Returns:
            str: Path to the downloaded video file

        Raises:
            CineNotesException: If video download or directory creation fails
        """
        try:
            logging.info(f"Starting video download from: {self.youtube_url}")
            yt = YouTube(self.youtube_url)
            # stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
            stream = yt.streams.get_highest_resolution()
            
            video_title = self._sanitize_filename(yt.title)
            self.video_dir = os.path.join(self.output_base_dir, video_title)
            os.makedirs(self.video_dir, exist_ok=True)
            
            self.video_filename = stream.download(output_path=self.video_dir, filename="video.mp4")
            logging.info(f"Video downloaded successfully to: {self.video_filename}")
            
            self.audio_dir = os.path.join(self.video_dir, "audio")
            self.frames_dir = os.path.join(self.video_dir, "frames")
            os.makedirs(self.audio_dir, exist_ok=True)
            os.makedirs(self.frames_dir, exist_ok=True)

            return self.video_filename
        except Exception as e:
            raise CineNotesException("Failed to download video", sys)

    def extract_audio(self, audio_filename="audio.wav"):
        """
        Extract audio from the downloaded video using ffmpeg.

        Args:
            audio_filename (str): Name for the output audio file (default: "audio.wav")

        Returns:
            str: Path to the extracted audio file

        Raises:
            ValueError: If no video has been downloaded
            CineNotesException: If audio extraction fails
        """
        try:
            if not self.video_filename:
                raise ValueError("No video has been downloaded. Call download_video() first.")

            audio_path = os.path.join(self.audio_dir, audio_filename)
            logging.info(f"Extracting audio to: {audio_path}")
            
            cmd = [
                "ffmpeg",
                "-i", self.video_filename,
                "-vn",
                "-acodec", "pcm_s16le",
                "-ar", "44100",
                "-ac", "2",
                audio_path,
                "-y"
            ]
            
            subprocess.run(cmd, check=True)
            logging.info("Audio extraction completed successfully")
            return audio_path
        except ValueError as ve:
            raise ve
        except Exception as e:
            raise CineNotesException("Failed to extract audio", sys)

    def extract_frames(self, image_prefix="frame_", image_format="jpg"):
        """
        Extract frames from the video using ffmpeg.

        Args:
            image_prefix (str): Prefix for frame filenames (default: "frame_")
            image_format (str): Format for frame images (default: "jpg")

        Returns:
            str: Path to the directory containing extracted frames

        Raises:
            ValueError: If no video has been downloaded
            CineNotesException: If frame extraction fails
        """
        try:
            if not self.video_filename:
                raise ValueError("No video has been downloaded. Call download_video() first.")
            
            logging.info(f"Extracting frames to: {self.frames_dir}")
            frame_pattern = os.path.join(self.frames_dir, f"{image_prefix}%05d.{image_format}")
            
            cmd = [
                "ffmpeg",
                "-i", self.video_filename,
                frame_pattern,
                "-y"
            ]
            
            subprocess.run(cmd, check=True)
            logging.info("Frame extraction completed successfully")
            return self.frames_dir
        except ValueError as ve:
            raise ve
        except Exception as e:
            raise CineNotesException("Failed to extract frames", sys)

    @staticmethod
    def _sanitize_filename(name: str) -> str:
        """
        Sanitize filename by removing/escaping invalid filename characters.

        Args:
            name (str): Original filename

        Returns:
            str: Sanitized filename
        """
        return "".join(c for c in name if c.isalnum() or c in (' ', '.', '_', '-')).strip()



In [3]:
youtube_url = "https://www.youtube.com/watch?v=qhomKbL-mHw"
processor = YouTubeVideoProcessor(youtube_url, output_base_dir="./my_videos")
processor.download_video()
# processor.extract_audio()
# processor.extract_frames()
print("Video processing complete!")

CineNotesException: Error occurred in python script: [C:\Users\Admin\AppData\Local\Temp\ipykernel_19424\1469822310.py] at line number: [64] with error: [Failed to download video]

In [4]:
! pip install yt-dlp

     ---------------------------------------- 3.2/3.2 MB 11.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import subprocess

youtube_url = "https://www.youtube.com/watch?v=qhomKbL-mHw"
output_path = "./my_videos/video.mp4"
subprocess.run(["yt-dlp", "-f", "mp4", "-o", output_path, youtube_url], check=True)


CompletedProcess(args=['yt-dlp', '-f', 'mp4', '-o', './my_videos/video.mp4', 'https://www.youtube.com/watch?v=qhomKbL-mHw'], returncode=0)

In [2]:
!pip install moviepy

  Using cached moviepy-2.1.1-py3-none-any.whl (123 kB)
  Using cached imageio-2.36.1-py3-none-any.whl (315 kB)
  Using cached proglog-0.1.10-py3-none-any.whl (6.1 kB)
  Using cached imageio_ffmpeg-0.5.1-py3-none-win_amd64.whl (22.6 MB)


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python310\\Scripts\\imageio_download_bin.exe' -> 'C:\\Python310\\Scripts\\imageio_download_bin.exe.deleteme'


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: C:\Python310\python.exe -m pip install --upgrade pip


In [ ]:
try:
    from moviepy.editor import VideoFileClip
    print("MoviePy imported successfully!")
except ImportError as e:
    print(f"ImportError: {e}")


In [3]:
!python

^C


In [4]:
!ffmpeg -version


'ffmpeg' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
!pip show ffmpeg

In [6]:
from imageio_ffmpeg import get_ffmpeg_exe

# Get the path to the FFmpeg executable
ffmpeg_path = get_ffmpeg_exe()

print(f"FFmpeg is installed at: {ffmpeg_path}")


FFmpeg is installed at: c:\Users\Admin\Documents\Data_Science\GenerativeAI\Repositories\CineNotes\venv\lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win64-v4.2.2.exe


In [9]:
# from moviepy.editor import VideoFileClip
from moviepy.video.io.VideoFileClip import VideoFileClip
# Test MoviePy with FFmpeg
print("MoviePy and FFmpeg are working!")


MoviePy and FFmpeg are working!
